In [11]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.style.use('classic')

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

In [13]:
import os
import cv2
from PIL import Image
import numpy as np

In [14]:
image_directory = '299aug-nor-macular/'
SIZE = 150
dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
label = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.

In [15]:
macular_degeneration_images = os.listdir(image_directory + 'macular-degeneration/')
for i, image_name in enumerate(macular_degeneration_images):    #Remember enumerate method adds a counter and returns the enumerate object
    
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'macular-degeneration/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

In [16]:
normal_images = os.listdir(image_directory + 'normal/')
for i, image_name in enumerate(normal_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'normal/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset = np.array(dataset)
label = np.array(label)

In [17]:
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical

In [18]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)

In [19]:
#Without scaling (normalize) the training may not converge. 
#Normalization is a rescaling of the data from the original range 
#so that all values are within the range of 0 and 1.
from tensorflow.keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

#Do not do one-hot encoding as it generates a shape of (num, 2)
#But the network expects an input of (num, 1) for the last layer for binary classification
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [27]:
# 2 conv and pool layers. with some normalization and drops in between.

INPUT_SHAPE = (SIZE, SIZE, 3)   #change to (SIZE, SIZE, 3)


model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Conv2D(128, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(1))
model.add(Activation('sigmoid'))  
#Do not use softmax for binary classification
#Softmax is useful for mutually exclusive classes, either cat or dog but not both.
#Also, softmax outputs all add to 1. So good for multi class problems where each
#class is given a probability and all add to 1. Highest one wins. 

In [28]:
#Sigmoid outputs probability. Can be used for non-mutually exclusive problems.
#But, also good for binary mutually exclusive (cat or not cat). 

model.compile(loss='binary_crossentropy',
              optimizer='adam',             #also try adam
              metrics=['accuracy'])


In [29]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
activation_24 (Activation)   (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 72, 72, 64)        36928     
_________________________________________________________________
activation_25 (Activation)   (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 36, 36, 64)       

In [31]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 32, 
                         verbose = 1, 
                         epochs = 20,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/20
200/200 [==============================] - 155s 773ms/step - loss: 0.5630 - accuracy: 0.6939 - val_loss: 0.5422 - val_accuracy: 0.7081
Epoch 2/20
200/200 [==============================] - 152s 758ms/step - loss: 0.4549 - accuracy: 0.7802 - val_loss: 0.4442 - val_accuracy: 0.7894
Epoch 3/20
200/200 [==============================] - 147s 733ms/step - loss: 0.4261 - accuracy: 0.7975 - val_loss: 0.4417 - val_accuracy: 0.7912
Epoch 4/20
200/200 [==============================] - 153s 768ms/step - loss: 0.3914 - accuracy: 0.8259 - val_loss: 0.4036 - val_accuracy: 0.8394
Epoch 5/20
200/200 [==============================] - 148s 741ms/step - loss: 0.3625 - accuracy: 0.8398 - val_loss: 0.3877 - val_accuracy: 0.8531
Epoch 6/20
200/200 [==============================] - 144s 722ms/step - loss: 0.3452 - accuracy: 0.8487 - val_loss: 0.4008 - val_accuracy: 0.8256
Epoch 7/20
200/200 [==============================] - 151s 755ms/step - loss: 0.3337 - accuracy: 0.8572 - val_loss: 0.3593 -

In [32]:
history = model.fit(X_train, 
                         y_train, 
                         batch_size = 32, 
                         verbose = 1, 
                         epochs = 5,      
                         validation_data=(X_test,y_test),
                         shuffle = False
                     )

Epoch 1/5
200/200 [==============================] - 137s 686ms/step - loss: 0.2629 - accuracy: 0.8814 - val_loss: 0.2796 - val_accuracy: 0.8775
Epoch 2/5
200/200 [==============================] - 136s 681ms/step - loss: 0.2588 - accuracy: 0.8844 - val_loss: 0.3050 - val_accuracy: 0.8531
Epoch 3/5
200/200 [==============================] - 136s 681ms/step - loss: 0.2596 - accuracy: 0.8814 - val_loss: 0.3145 - val_accuracy: 0.8494
Epoch 4/5
200/200 [==============================] - 142s 710ms/step - loss: 0.2417 - accuracy: 0.8878 - val_loss: 0.2906 - val_accuracy: 0.8612
Epoch 5/5
200/200 [==============================] - 137s 687ms/step - loss: 0.2367 - accuracy: 0.8936 - val_loss: 0.2762 - val_accuracy: 0.8681
